# **Libraries**

In [1]:
!pip install timm

In [2]:
!pip install pytorch_tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.9 MB/s eta 0:00:00


In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [4]:
# Data Processing n' Visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Compute
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Data
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Random
import os
import random as rand
import timm

In [28]:
torch.cuda.empty_cache()

In [5]:
def set_seed(seed):
  rand.seed(seed)
  np.random.seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed = 59
set_seed(59)

--------------------
# **Data Sample**

In [6]:
data_dir = '/kaggle/input/wharton-basketball-dataset/games_2022.xlsx'

In [7]:
df = pd.read_excel(data_dir)
print(f"Dataset Type: {type(df)}")

Dataset Type: <class 'pandas.core.frame.DataFrame'>


In [8]:
df
df_ts = df
df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,game_id,game_date,team,FGA_2,FGM_2,FGA_3,FGM_3,FTA,FTM,AST,...,largest_lead,notD1_incomplete,OT_length_min_tot,rest_days,attendance,tz_dif_H_E,prev_game_dist,home_away,home_away_NS,travel_dist
0,game_2022_2011,2021-12-30,georgia_lady_bulldogs,50,22,11,5,6,3,14,...,1.0,False,NaN,9.0,3241.0,0.0,0.0,home,1,0.0
1,game_2022_2011,2021-12-30,lsu_tigers,50,24,11,4,15,8,15,...,14.0,False,NaN,3.0,3241.0,0.0,824.0,away,-1,824.0
2,game_2022_2012,2021-12-30,missouri_tigers,43,18,15,7,16,13,10,...,8.0,False,5.0,8.0,6139.0,0.0,371.0,home,1,0.0
3,game_2022_2012,2021-12-30,south_carolina_gamecocks,55,23,21,6,9,5,15,...,6.0,False,5.0,9.0,6139.0,0.0,1154.0,away,-1,1154.0
4,game_2022_2013,2021-12-30,tennessee_lady_volunteers,41,20,15,4,15,10,16,...,19.0,False,NaN,3.0,8124.0,0.0,0.0,home,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,game_2022_4795,2022-03-01,xavier_musketeers,44,19,12,2,12,9,15,...,0.0,False,NaN,2.0,156.0,0.0,0.0,home,1,0.0
10434,game_2022_4968,2022-03-04,harvard_crimson,30,19,39,15,10,2,20,...,40.0,False,NaN,13.0,373.0,0.0,378.0,away,-1,173.0
10435,game_2022_4968,2022-03-04,dartmouth_big_green,32,15,26,6,7,4,7,...,3.0,False,NaN,6.0,373.0,0.0,479.0,home,1,0.0
10436,game_2022_5067,2022-03-06,harvard_crimson,38,13,35,6,10,9,11,...,0.0,False,NaN,2.0,757.0,0.0,173.0,home,1,0.0


----------------------------
# **Data Preprocessing**

In [9]:
df_ts = df_ts.drop(columns = ['OT_length_min_tot', 'attendance', 'tz_dif_H_E', 'opponent_team_score', 
                        'team_score', 'home_away', 'notD1_incomplete', 'largest_lead'])
df_ts = df_ts.dropna()
df_ts['home_away_NS'] = df_ts['home_away_NS'].replace({
    1: 1, -1: 0, 0: 2
})

from sklearn.preprocessing import MinMaxScaler

# List of columns to normalize
stats_to_normalize = ['FGA_2', 'FGM_2', 'FGA_3', 'FGM_3', 
                      'FTA', 'FTM', 'AST', 'BLK', 'STL', 'TOV', 
                      'TOV_team', 'DREB', 'OREB', 'F_tech', 'F_personal', 
                      'rest_days', 'prev_game_dist', 'travel_dist']

# Initialize MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

# Apply MinMaxScaler only to the selected stats
df_ts[stats_to_normalize] = scaler.fit_transform(df_ts[stats_to_normalize])

print(df_ts.head())  # Check normalized values

          game_id  game_date                       team     FGA_2     FGM_2  \
0  game_2022_2011 2021-12-30      georgia_lady_bulldogs  0.661290  0.513514   
1  game_2022_2011 2021-12-30                 lsu_tigers  0.661290  0.567568   
2  game_2022_2012 2021-12-30            missouri_tigers  0.548387  0.405405   
3  game_2022_2012 2021-12-30   south_carolina_gamecocks  0.741935  0.540541   
4  game_2022_2013 2021-12-30  tennessee_lady_volunteers  0.516129  0.459459   

      FGA_3     FGM_3       FTA    FTM       AST  ...       TOV  TOV_team  \
0  0.196078  0.227273  0.125000  0.075  0.361111  ...  0.390244  0.000000   
1  0.196078  0.181818  0.312500  0.200  0.388889  ...  0.292683  0.333333   
2  0.274510  0.318182  0.333333  0.325  0.250000  ...  0.146341  0.166667   
3  0.392157  0.272727  0.187500  0.125  0.388889  ...  0.146341  0.000000   
4  0.274510  0.181818  0.312500  0.250  0.416667  ...  0.317073  0.166667   

       DREB     OREB  F_tech  F_personal  rest_days  prev_game

In [10]:
display(df_ts)
print(df_ts.columns)

,game_id,game_date,team,FGA_2,FGM_2,FGA_3,FGM_3,FTA,FTM,AST,...,TOV,TOV_team,DREB,OREB,F_tech,F_personal,rest_days,prev_game_dist,home_away_NS,travel_dist
0,game_2022_2011,2021-12-30,georgia_lady_bulldogs,0.661290,0.513514,0.196078,0.227273,0.125000,0.075,0.361111,...,0.390244,0.000000,0.410256,0.34375,0.0,0.451613,0.210526,0.000000,1,0.000000
1,game_2022_2011,2021-12-30,lsu_tigers,0.661290,0.567568,0.196078,0.181818,0.312500,0.200,0.388889,...,0.292683,0.333333,0.410256,0.34375,0.0,0.096774,0.052632,0.105722,0,0.121105
2,game_2022_2012,2021-12-30,missouri_tigers,0.548387,0.405405,0.274510,0.318182,0.333333,0.325,0.250000,...,0.146341,0.166667,0.564103,0.18750,0.0,0.225806,0.184211,0.047601,1,0.000000
3,game_2022_2012,2021-12-30,south_carolina_gamecocks,0.741935,0.540541,0.392157,0.272727,0.187500,0.125,0.388889,...,0.146341,0.000000,0.461538,0.62500,0.0,0.354839,0.210526,0.148063,0,0.169606
4,game_2022_2013,2021-12-30,tennessee_lady_volunteers,0.516129,0.459459,0.274510,0.181818,0.312500,0.250,0.416667,...,0.317073,0.166667,0.641026,0.37500,0.0,0.290323,0.052632,0.000000,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,game_2022_4795,2022-03-01,xavier_musketeers,0.564516,0.432432,0.215686,0.090909,0.250000,0.225,0.388889,...,0.195122,0.000000,0.410256,0.25000,0.0,0.258065,0.026316,0.000000,1,0.000000
10434,game_2022_4968,2022-03-04,harvard_crimson,0.338710,0.432432,0.745098,0.681818,0.208333,0.050,0.527778,...,0.097561,0.000000,0.487179,0.31250,0.0,0.129032,0.315789,0.048499,0,0.025426
10435,game_2022_4968,2022-03-04,dartmouth_big_green,0.370968,0.324324,0.490196,0.272727,0.145833,0.100,0.166667,...,0.463415,0.333333,0.512821,0.31250,0.0,0.129032,0.131579,0.061458,1,0.000000
10436,game_2022_5067,2022-03-06,harvard_crimson,0.467742,0.270270,0.666667,0.272727,0.208333,0.225,0.277778,...,0.219512,0.000000,0.358974,0.56250,0.0,0.516129,0.026316,0.022197,1,0.000000


Index(['game_id', 'game_date', 'team', 'FGA_2', 'FGM_2', 'FGA_3', 'FGM_3',
       'FTA', 'FTM', 'AST', 'BLK', 'STL', 'TOV', 'TOV_team', 'DREB', 'OREB',
       'F_tech', 'F_personal', 'rest_days', 'prev_game_dist', 'home_away_NS',
       'travel_dist'],
      dtype='object')


In [11]:
Data = {}
X = []
y = []
start_col = 'FGA_2'
end_col = 'F_personal'

start_col_nx = 'rest_days'
end_col_nx = 'travel_dist'
# Loop through all Teams in Dataset
for i ,team in enumerate(df_ts['team'].unique()):
    team_data = df_ts[df_ts['team'] == team]
    for idx in range(len(team_data) - 1):
        # Features = past game (FGA_2 to F_personal) + current game stats (rest_days to travel_dist)
        past_game = team_data.iloc[idx]
        next_game = team_data.iloc[idx + 1]
        
        past_stats = past_game.loc[start_col : end_col].values
        next_stats = next_game.loc[start_col_nx:end_col_nx].values

        # We predict next game FGA_2 to F_personal
        label = next_game.loc[start_col:end_col].values

        if idx == 0 and i == 0:
            combined_stats_x = np.concatenate((past_stats, next_stats))
            print(combined_stats_x)
            print(f"Type of combined_stats: {type(combined_stats_x)}")
            print(f"First Index: {combined_stats_x[0]}")
            print("-"*59)
            print(f"Label: {label}")
            print(f"Type of Label: {type(label)}")
            print(f"First Index: {label[0]}")

        combined_stats = np.concatenate((past_stats, next_stats))
        X.append(combined_stats)
        y.append(label)

[0.6612903225806451 0.5135135135135136 0.19607843137254904
 0.2272727272727273 0.125 0.07500000000000001 0.36111111111111105
 0.38888888888888884 0.25925925925925924 0.3902439024390244 0.0
 0.41025641025641024 0.34375 0.0 0.4516129032258064 0.05263157894736842
 0.06209905055170644 0 0.07113462669018225]
Type of combined_stats: <class 'numpy.ndarray'>
First Index: 0.6612903225806451
-----------------------------------------------------------
Label: [0.7580645161290323 0.6216216216216217 0.09803921568627451
 0.045454545454545456 0.47916666666666663 0.45 0.25 0.1111111111111111
 0.5555555555555556 0.3658536585365854 0.3333333333333333
 0.2564102564102564 0.5 0.0 0.3548387096774194]
Type of Label: <class 'numpy.ndarray'>
First Index: 0.7580645161290323


In [12]:
print(f"First 5 Games Features: {X[:5]}")
print(f"Number of Features: {len(X[0])}")

print("-"*59)
print(f"First 5 Games Labels: {y[:5]}")
print(f"Number of Features to Predict: {len(y[0])}")

print(f"Type: {type(X), type(y)}")

First 5 Games Features: [array([0.6612903225806451, 0.5135135135135136, 0.19607843137254904,
       0.2272727272727273, 0.125, 0.07500000000000001,
       0.36111111111111105, 0.38888888888888884, 0.25925925925925924,
       0.3902439024390244, 0.0, 0.41025641025641024, 0.34375, 0.0,
       0.4516129032258064, 0.05263157894736842, 0.06209905055170644, 0,
       0.07113462669018225], dtype=object), array([0.7580645161290323, 0.6216216216216217, 0.09803921568627451,
       0.045454545454545456, 0.47916666666666663, 0.45, 0.25,
       0.1111111111111111, 0.5555555555555556, 0.3658536585365854,
       0.3333333333333333, 0.2564102564102564, 0.5, 0.0,
       0.3548387096774194, 0.07894736842105263, 0.1216320246343341, 0,
       0.06819517930629042], dtype=object), array([0.6612903225806451, 0.6216216216216217, 0.1764705882352941,
       0.09090909090909091, 0.41666666666666663, 0.45,
       0.38888888888888884, 0.2222222222222222, 0.2962962962962963,
       0.41463414634146345, 0.1666666666

In [13]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# **Dataset**

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=42)

print(X_train.shape)  # Should be (num_samples, num_features) → (N, 19)
print(y_train.shape)  # Should be (num_samples, num_outputs) → (N, 15)
print(f"Type of X_train and X_test: {type(X_train)} | {type(X_test)}")
print(X_train.dtype, y_train.dtype)
print(X_val.dtype, y_val.dtype)
print(X_test.dtype, y_test.dtype)

(6139, 19)
(6139, 15)
Type of X_train and X_test: <class 'numpy.ndarray'> | <class 'numpy.ndarray'>
float32 float32
float32 float32
float32 float32


In [15]:
import torch
from torch.utils.data import Dataset

class bkb_dataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature = torch.tensor(self.data[idx], dtype=torch.float32)  # Use float32
        label = torch.tensor(self.label[idx], dtype=torch.float32)  # Use float32

        return {"input_ids": feature, "labels": label}

In [16]:
train_set = bkb_dataset(
    X_train,
    y_train,
)

val_set = bkb_dataset(
    X_val,
    y_val,
)

test_set = bkb_dataset(
    X_test,
    y_test,
)
print(f"Length of train_set: {len(train_set)}")
print(f"Length of test_set: {len(test_set)}")
print(f"Length of val_set: {len(val_set)}")

Length of train_set: 6139
Length of test_set: 1763
Length of val_set: 869


In [17]:
train_batch = 256
test_batch = 32

train_loader = DataLoader(
    train_set,
    batch_size = train_batch,
    shuffle = True
)

val_loader = DataLoader(
    val_set,
    batch_size = test_batch,
    shuffle = False
)

test_loader = DataLoader(
    test_set,
    batch_size = test_batch,
    shuffle = False
)
print(f"Length train_loader: {len(train_loader)}")
print(f"Length test_loader: {len(test_loader)}")
print(f"Length val_loader: {len(val_loader)}")

Length train_loader: 24
Length test_loader: 56
Length val_loader: 28


# **Model**

In [18]:
from huggingface_hub import login

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HF_TOKEN")

# Login to Hugging Face
login(HUGGINGFACE_TOKEN)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
import torch.nn.functional as F

class FeatureGrouping(nn.Module):
    def __init__(self, num_features=19, max_groups=3, embed_dim=4, output_dim=15, resnet_model="resnet50", finetune = False):
        super().__init__()
        self.num_features = num_features
        self.max_groups = max_groups
        self.embed_dim = embed_dim
        self.output_dim = output_dim

        # Embedding for home_away (3 categories: Home, Away, Neutral)
        self.home_away_embed = nn.Embedding(3, embed_dim)

        # Adjust feature count after embedding replacement
        self.adjusted_num_features = num_features - 1 + embed_dim  # 22 total features

        # Calculate the maximum features per group to ensure consistent dimensions
        self.max_features_per_group = self.adjusted_num_features

        # Learnable logits for feature assignment
        self.assignment_logits = nn.Parameter(torch.randn(self.adjusted_num_features, max_groups))

        # Self-Attention layers for each possible number of groups
        self.attention_layers = nn.ModuleDict({
            f"attn_{g}": nn.MultiheadAttention(
                embed_dim=self.max_features_per_group,
                num_heads=1,
                batch_first=True
            )
            for g in range(1, max_groups + 1)
        })

        # Reduce channels before ResNet
        self.channel_reducer = nn.Conv2d(in_channels=max_groups, out_channels=3, kernel_size=1)

        # Pretrained ResNet model
        self.resnet = timm.create_model(resnet_model, pretrained=True)
        in_features = self.resnet.get_classifier().in_features
        self.resnet.reset_classifier(0)

        # Final regression head
        self.fc = nn.Linear(in_features, output_dim)

    def forward(self, x):
        batch_size = x.shape[0]

        # Extract home_away index and convert to embeddings
        home_away_idx = x[:, 17].long().clamp(0, 2)
        home_away_embed = self.home_away_embed(home_away_idx)
        x = torch.cat([x[:, :17], home_away_embed, x[:, 18:]], dim=1)

        # Hard feature assignment
        assignment_hard = torch.argmax(self.assignment_logits, dim=1)

        all_group_outputs = []

        # Process different group configurations
        for num_groups in range(1, self.max_groups + 1):
            # Split features into groups
            groups = []
            features_per_group = self.adjusted_num_features // num_groups
            
            for g in range(num_groups):
                start_idx = g * features_per_group
                end_idx = min(start_idx + features_per_group, self.adjusted_num_features)
                group_features = x[:, start_idx:end_idx]
                
                # Pad to match max_features_per_group
                if group_features.shape[1] < self.max_features_per_group:
                    pad_size = self.max_features_per_group - group_features.shape[1]
                    padding = torch.zeros(batch_size, pad_size, device=x.device)
                    group_features = torch.cat([group_features, padding], dim=1)
                
                groups.append(group_features)

            # Process each group with attention
            processed_groups = []
            for g in range(num_groups):
                group_features = groups[g].unsqueeze(1)
                attn_output, _ = self.attention_layers[f"attn_{num_groups}"](
                    group_features, group_features, group_features)
                processed_groups.append(attn_output)

            # Combine processed groups
            group_output = torch.cat(processed_groups, dim=1)
            
            # Pad to match max_groups if necessary
            if num_groups < self.max_groups:
                padding = torch.zeros(
                    batch_size,
                    self.max_groups - num_groups,
                    self.max_features_per_group,
                    device=x.device
                )
                group_output = torch.cat([group_output, padding], dim=1)
            
            all_group_outputs.append(group_output)

        # Stack all configurations
        x_final = torch.stack(all_group_outputs, dim=1)  # [B, max_groups, max_groups, Features]
        
        # Reshape for channel reducer
        x_final = x_final.mean(dim=1)  # [B, max_groups, Features]
        x_final = x_final.permute(0, 2, 1)  # [B, Features, max_groups]
        x_final = x_final.mean(dim=1).unsqueeze(-1).unsqueeze(-1)  # [B, max_groups, 1, 1]
        
        # Apply channel reduction
        x_final = self.channel_reducer(x_final)  # [B, 3, 1, 1]
        
        # Prepare for ResNet
        x_final = x_final.expand(-1, -1, 224, 224)  # [B, 3, 224, 224]

        # Process through ResNet and final layer
        x_final = self.resnet(x_final)
        output = self.fc(x_final)

        return output

batch_size = 1
x = torch.randn(batch_size, 19)  # Example input features
model = FeatureGrouping(resnet_model="resnet50")  # Using ResNet50
output = model(x)
print(output.shape)  # Expected: (B, 15)
print(output)

torch.Size([1, 15])
tensor([[ 0.0303, -0.0341, -0.1230,  0.0756, -0.0159,  0.0239,  0.0422,  0.0751,
          0.0192, -0.1020,  0.0192, -0.0512,  0.0525,  0.0041,  0.0098]],
       grad_fn=<AddmmBackward0>)


In [21]:
from transformers import PreTrainedModel, PretrainedConfig

class FeatureGroupingConfig(PretrainedConfig):
    model_type = "feature_grouping"

    def __init__(self, num_features=19, output_dim=15, **kwargs):
        super().__init__(**kwargs)
        self.num_features = num_features
        self.output_dim = output_dim

class FeatureGroupingModel(PreTrainedModel):
    config_class = FeatureGroupingConfig

    def __init__(self, config):
        super().__init__(config)
        self.model = FeatureGrouping(
            num_features=config.num_features, 
            output_dim=config.output_dim
        )

    def forward(self, input_ids, labels=None):
        output = self.model(input_ids)

        loss = None
        if labels is not None:
            loss = F.mse_loss(output, labels)  # Mean Squared Error for regression

        return {"loss": loss, "logits": output} if loss is not None else {"logits": output}

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [25]:
import evaluate

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.array(predictions)
    labels = np.array(labels)

    mse_per_feature = ((predictions - labels) ** 2).mean(axis=0)  # (15,)
    mse_mean = mse_per_feature.mean()  # Overall MSE

    print(f"\n📢 Epoch MSE: {mse_mean:.4f}")  # Print MSE for each epoch

    return {"mse_mean": mse_mean}

In [26]:
from transformers import TrainerCallback

class LossLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"📢 Epoch {state.epoch:.0f}: Train Loss = {logs['loss']:.4f}")

In [32]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./basketball_model",
    learning_rate = 1e-5,
    eval_strategy="epoch",  # Evaluate at each epoch
    save_strategy="epoch",
    logging_strategy="epoch",  # Log every epoch
    logging_dir="./logs",
    logging_steps=1,  # Log at every step
    per_device_train_batch_size=128,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
    report_to="none",
    push_to_hub=True,
    optim = 'adamw_torch'
)

# Initialize Model
model = FeatureGroupingModel(FeatureGroupingConfig())

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set
    #callbacks=[LossLoggerCallback()]  # Add the callback here
)

# Train the Model
trainer.train()

# Save Model
trainer.save_model("./basketball_model")

# Save Model to Hugging Face Hub
trainer.push_to_hub("KanWasTaken/RandModel")

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.075600,0.075402
2,0.022600,0.027883
3,0.018400,0.034453


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


KeyboardInterrupt: 

# **Test**

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.3, random_state=42)
X_val1, X_test1, y_val1, y_test1 = train_test_split(X_test1, y_test1, test_size=0.33, random_state=42)

print(X_train1.shape)  # Should be (num_samples, num_features) → (N, 19)
print(y_train1.shape)  # Should be (num_samples, num_outputs) → (N, 15)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
# Wrap XGBRegressor to handle multi-output regression
model = MultiOutputRegressor(XGBRegressor(
    objective='reg:squarederror',  
    n_estimators=100,  
    learning_rate=0.1  
))

# Train model
model.fit(X_train1, y_train1)  

y_pred = model.predict(X_test1)
print(y_pred.shape)  # Should be (N_test, 15)

from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test1, y_pred, multioutput='raw_values')  # RMSE for each label
print("RMSE per output:", rmse)
print("Average RMSE:", rmse.mean())  # Average RMSE across all 15 labels

In [ ]:
X_pred_home = model.predict(np.array([X_train1[0]]))  # Ensures a 2D shape (1, 19)
X_pred_away = model.predict(np.array([X_train1[1]]))
print(X_pred_home)
print(X_pred_away)

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor

X_train1 = X_train1.astype(np.float32)
y_train1 = y_train1.astype(np.float32)
X_val1 = X_val1.astype(np.float32)
y_val1 = y_val1.astype(np.float32)

X_test1 = X_test1.astype(np.float32)
y_test1 = y_test1.astype(np.float32)

if y_test1.ndim == 1:
    y_test1 = y_test1.reshape(-1, 1)

if y_train1.ndim == 1:
    y_train1 = y_train1.reshape(-1, 1)
if y_val1.ndim == 1:
    y_val1 = y_val1.reshape(-1, 1)
    
model = TabNetRegressor()
model.fit(X_train1, y_train1, eval_set=[(X_val1, y_val1)])
preds = model.predict(X_test1)
print(preds)